### 한국어 분석을 위한 환경 구성

- Mac도 동일하게 진행

[vs 2015설치, Xcode설치]
   - 제공한 설치파일을 실행하고 사용자 지정을 선택하여 프로그래밍 언어에 있는 visual c++을 추가하여 설치
   
[jdk 설치]
   - 기본설정으로 설치
   
[jdk 환경변수 설정]
   - 내 PC 우클릭 속성 -> 환경변수를 클릭
   - 새로만들기를 눌러 다음과 같이 작성한다.
   - 변수 이름 : JAVA_HOME
   - 변수 값 : jdk가 있는 경로
   - 주피터를 완전히 종료하고 다시 실행한다.
 
[필요 라이브러리 설치]
- pip install konlpy==0.5.1
- pip install jpype1
- pip install jpype1-py3

### 워드 클라우드를 사용하기 위한 설정
- pip install pytagcloud
- pip install pygame
- pip install simplejson

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns

import requests
from bs4 import BeautifulSoup
import re
import time
import os

# 그래프 설정
# plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

In [2]:
from konlpy.tag import Okt
from collections import Counter
import pytagcloud
from IPython.display import Image
import re

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


### 데이터를 읽어온다.

In [3]:
df = pd.read_csv('wiki.csv')
df.head()

,title,content
0,대한건설기계안전관리원,대한민국의 공공기관 [ 펼치기 · 접기 ]시장형 공기업산업통상자원부강원랜드 | 한국...
1,틀:김천 상무,김천 상무 2021 시즌 스쿼드 [ 펼치기 · 접기 ]김천 상무 2021 시즌 스...
2,에이레네,은(는) 여기로 연결됩니다. 프린세스의 등장인물에 대한 내용은 에이레네 로디트 문서...
3,민영(브레이브걸스),민영유정은지유나 [ 전 멤버 ]은영서아예진유진혜란하윤 [ 관련 문서 ]기수 1기 ｜...
4,베스카,제국제[1] 베스카 주괴What exquisite craftmanship. It i...


In [4]:
# 텍스트 정제 함수 : 한글 이외의 문자는 전부 제거한다.
def text_cleaning(text):
    # 한글 정규식 정의(띄어쓰기, ㄱ~ㅣ, 가 ~ 힣)에 해당하는 것만 가져온다
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]+')
    # 정제한다. (정규식에 해당하지 않는 글자는 길이가 0인 문자열로 변환한다.)
    result = hangul.sub('',text)
    return result

In [5]:
# 결측치 확인
df.isna().sum()

title      0
content    2
dtype: int64

In [6]:
# 결측치 제거
df.dropna(inplace=True)

In [7]:
# 결측치 확인
df.isna().sum()

title      0
content    0
dtype: int64

In [8]:
# 모든 특성에 대해 정제한다.
df['title'] = df['title'].apply(lambda x : text_cleaning(x))
df['content'] = df['content'].apply(lambda x : text_cleaning(x))

In [9]:
df

,title,content
0,대한건설기계안전관리원,대한민국의 공공기관 펼치기 접기 시장형 공기업산업통상자원부강원랜드 한국가스공사...
1,틀김천 상무,김천 상무 시즌 스쿼드 펼치기 접기 김천 상무 시즌 스쿼드주의등번호국적포지...
2,에이레네,은는 여기로 연결됩니다 프린세스의 등장인물에 대한 내용은 에이레네 로디트 문서를의 ...
3,민영브레이브걸스,민영유정은지유나 전 멤버 은영서아예진유진혜란하윤 관련 문서 기수 기 팬덤 피어...
4,베스카,제국제 베스카 주괴 이 얼마나 정교한 솜씨인가 대를 이은...
...,...,...
91,반월동화성,경기도 화성시 행정구역 경기도 화성시 행정구역 펼치기 접기 읍봉담읍우정읍향남읍...
92,노돌리,멤버 펼치기 접기 원년 멤버우왁굳개복어노돌리노지로복레비레알마틴렘쨩무결조마문조매력...
93,브랜든 나이트야구,브랜든 나이트의 수상 경력 역대 등번호 펼치기 접기 일본시리즈 우승 반지 ...
94,년 미얀마 민주화 운동,이 문서는이 문단은 토론을 통해 경과 문단을 분리하지 않기으로 합의되었습니다 합의된...


In [10]:
# 말뭉치 생성 (각 컬럼의 문자열들을 하나로 합쳐준다.)
# title을 리스트로 받아 길이가 0인 문자열과 합쳐준다.
title_corpus = ''.join(df['title'].tolist())
content_corpus = ''.join(df['content'].tolist())

In [ ]:
# 형태소 분석
tagger = Okt()
title_nouns = tagger.nouns(title_corpus)
content_nouns = tagger.nouns(content_corpus)

print(title_nouns)
print(content_nouns)

In [ ]:
# 각 단어들의 빈도수를 구한다.
title_count = Counter(title_nouns)
content_count = Counter(content_nouns)

print(title_count)
print(content_count)

### 키워드 다듬기

In [ ]:
 
# 2글자 이상인 단어를 담을 딕셔너리
title_dict = {}

# 워드 클라우드는 두 글자 이상만 가능하므로 한글자는 제거한다.
for key in title_count:
    # 글자 길이가 1보다 크다면 딕셔너리에 담아준다.
    if len(key) > 1:
        title_dict[key] = title_count[key]

# 2글자 이상인 단어를 담을 딕셔너리
content_dict = {}

# 워드 클라우드는 두 글자 이상만 가능하므로 한글자는 제거한다.
for key in content_dict:
    # 글자 길이가 1보다 크다면 딕셔너리에 담아준다.
    if len(key) > 1:
        content_dict[key] = content_count[key]

# print(len(title_dict))
# print(len(content_dict))

remove_title_count = Counter(title_dict)
remove_content_count = Counter(content_dict)

# print(len(remove_title_count))
# print(len(remove_content_count))

In [ ]:
# 불용어 제거
# 불용어 (stopwords) : 분석시 필요하지 않는 단어들을 의미
# 불용어를 불러온다.
with open('data10/korean_stopwords.txt', encoding='utf-8') as fp:
    stopwords = fp.readlines()

# \n(공백) 제거
stopwords = [x.strip() for x in stopwords]

In [ ]:
# 불용어를 제외한 나머지를 새롭게 담아준다.
content_dict = {}
for key in remove_content_count:
    # 현재 글자가 불용어에 포함되어 있지 않다면..
    if key not in stopwords:
        content_dict[key] - remove_content_count[key]

title_dict = {}
for key in remove_title_count:
    # 현재 글자가 불용어에 포함되어 있지 않다면..
    if key not in stopwords:
        title_dict[key] - remove_title_count[key]
        
        
# print(len(remove_title_count))
# print(len(remove_content_count))

### 워드클라우드로 시각화
- 한글 폰트 설치
- https://hangeul.naver.com/webfont/NanumGothic/NanumGothic.ttf 설치
- 내려받은 폰트를 해당 경로에 넣어준다.
    - c드라이브 -> 사용자 -> anaconda3 -> lib -> site-packages -> pytagcloud -> fonts
    
- font.json 파일에 폰트를 등록해준다.
    <pre>
    - {
            "name" : "NanumGothic",
            "ttf" : "NanumGothic.ttf",
            "web" : "http://fonts.googleapis.com/css?family=Nanum+Gothic"
      }
    </pre>

In [ ]:
# title에서 가장 많이 나온 단어 40개를 추출한다.
ranked_title_tags = remove_title_count.most_common(40)
b
# 워드 클라우드를 위한 단어 사전을 생성한다.
title_taglist = pytagcloud.make_tags(remove_title_count)

# 워드 클라우드 이미지를 생성한다.
pytagcloud.create_tag_image(title_taglist, 'title_word.jpg', size=(900,600),
                            fontname'NanumGothic', rectangular=False)

In [ ]:
# content에서 가장 많이 사용하는 단어 40개를 추출한다.
ranked_content_tags = remove_content_count.most_common(40)

# 워드 클라우드를 위한 단어 사전을 생성한다.
content_taglist = pytagcloud.make_tags(ranked_content_tags)

# 워드 클라우드 이미지를 생성한다.
pytagcloud.create_tag_image(content_taglist, 'content_word.jpg'
                            , size=(900, 600), fontname='NanumGothic'
                            , rectangular=False)

In [ ]:
print('title')
Image(filename='title_word.jpg')

In [ ]:
print('content')
Image(filename='content_word.jpg')